In [65]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [103]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [67]:
df = pd.read_sql('SELECT * FROM mcrecos', con=database_connection)
#del df['srno']
del df['index']


In [68]:
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [69]:
df.drop_duplicates(inplace = True)
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [70]:
df['Call_Time_normal'] =  pd.to_datetime(df['Call_Time_normal'], format='%Y-%m-%d')
df['Call_Time_normal'] = df['Call_Time_normal'].dt.date
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05
5211,HDIL,83.0,Rajat Bose,2016-01-04
5212,DLF,125.5,Rajat Bose,2016-01-04
5213,Vinati Organics,555.0,Karvy,2016-01-01


In [71]:
start_date = '2016-01-01'
end_date = '2016-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [72]:
mask = (df['Call_Time_normal'] > date_object1) & (df['Call_Time_normal'] <= date_object2)

# 2016 data

In [73]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [74]:
df1

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,"FIEM Industries, see 23% CAGR",1315.0,Religare,2016-12-30
1,Petronet LNG,411.0,Motilal Oswal,2016-12-28
2,Karnataka Bank,136.0,Arihant Capital,2016-12-28
3,ITC,306.0,ICICI Securties,2016-12-28
4,West Coast Paper Mills,169.0,AUM Capital,2016-12-26
...,...,...,...,...
1242,Sangam,355.0,Firstcall,2016-01-05
1243,Supreme Industries,795.0,Karvy,2016-01-05
1244,Samkrg Pistons,210.0,Firstcall,2016-01-05
1245,HDIL,83.0,Rajat Bose,2016-01-04


In [75]:
print("No of Brokers in 2016 :",df1['Analyst_or_Cmp'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 95


## CLEANING

In [76]:
dfsymbols = pd.read_sql("SELECT SYMBOL,recoName FROM symbolsmappedmc", con=database_connection)
dfsymbols

,SYMBOL,recoName
0,SIYSIL,siyaramsilkmills
1,SIYSIL,siyaramsilk
2,JKLAKSHMI,jklakshmicement
3,JKLAKSHMI,jklakshmicements
4,JKLAKSHMI,jklakshmicement
...,...,...
1169,None,uplondeclines
1170,None,jetairwaysondips
1171,None,jenburktpharma
1172,None,samkrgpistons


In [77]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'Cmp_Name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    appenddf = dfsymbols.loc[dfsymbols['recoName']==sym]
    del appenddf['recoName']
    df3 = df3.append(appenddf,ignore_index=True)

In [78]:
len(df1)

1247

In [79]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Data columns (total 1 columns):
SYMBOL    901 non-null object
dtypes: object(1)
memory usage: 10.2+ KB


In [80]:
df3["SYMBOL"].fillna(method='ffill', inplace = True) 

In [81]:
df3=df3.replace("None",np.nan)

In [82]:
df3

,SYMBOL
0,None
1,PETRONET
2,KTKBANK
3,KTKBANK
4,WSTCSTPAPR
...,...
1288,SANGAMIND
1289,SUPREMEIND
1290,SUPREMEIND
1291,SUPREMEIND


## SYMBOLS ADDED TO TABLE

In [83]:
df4 =pd.concat([df1,df3],axis=1)

In [84]:
df4 = df4.dropna()

In [85]:
df4

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal,SYMBOL
1,Petronet LNG,411.0,Motilal Oswal,2016-12-28,PETRONET
2,Karnataka Bank,136.0,Arihant Capital,2016-12-28,KTKBANK
3,ITC,306.0,ICICI Securties,2016-12-28,KTKBANK
4,West Coast Paper Mills,169.0,AUM Capital,2016-12-26,WSTCSTPAPR
5,Tube Investments,719.0,AUM Capital,2016-12-26,TIINDIA
...,...,...,...,...,...
1242,Sangam,355.0,Firstcall,2016-01-05,SANDESH
1243,Supreme Industries,795.0,Karvy,2016-01-05,ATULAUTO
1244,Samkrg Pistons,210.0,Firstcall,2016-01-05,ATULAUTO
1245,HDIL,83.0,Rajat Bose,2016-01-04,INDUSINDBK


In [86]:
df4.drop(df4.loc[df4['SYMBOL']=='IFGLEXPOR'].index, inplace=True)

In [87]:
uniqueBrokers = df4.Analyst_or_Cmp.unique()
uniqueBrokers

array(['Motilal Oswal', 'Arihant Capital', 'ICICI Securties',
       'AUM Capital', 'Edelweiss', 'ICICI Direct', 'Geojit BNP Paribas',
       'Axis Securities', 'Axis Direct', 'Centrum', 'Prabhudas Lilladher',
       'Religare', 'KR Choksey', 'HDFC Securities', 'HDFC Sec',
       'Dynamic Levels', 'GEPL Capital', 'Yogesh Mehta', 'Hemant Thukral',
       'Ajcon Global', 'Aum Capital', 'Way2Wealth', 'IIFL',
       'SPA Research', 'Anand Rathi', 'Khambatta Securities',
       'CD Equisearch', 'East India Securities', 'SPA Financial', 'Karvy',
       'Dolat Capital', 'Reliance Securities', 'LKP Securities',
       'Nirmal Bang', 'Yes Securities', 'Rahul Mohindar',
       'Shahina Mukadam', 'Mitesh Thacker', 'Antique Stock Broking',
       'Ashwani Gujral', 'Dynamic Level', 'Antique', 'Hem Securities',
       'Investment Imperative', 'Amit Gupta', 'Systematix', 'EIS',
       'Rajat Bose', 'KRChoksey', 'Prabhudas Lillader',
       'Manoj Murlidharan', 'Chandan Taparia', 'Emkay', 'Manoj Beher

In [88]:
#uniqueBrokers = uniqueBrokers[:1]

## PERFORMANCE DEDUCTION ALGORITHM

In [89]:
cnx = mysql.connector.connect(user="root", password='yash', host="127.0.0.1", port=3306, database='mainschema')
sql_select_Query = """SELECT Closeval FROM nse2016 WHERE Date = '2016-11-17 ' AND Symbol = 'IFGLEXPOR'"""
cursor = cnx.cursor()
cursor.execute(sql_select_Query)
record = cursor.fetchall()

In [90]:
record

[]

In [91]:
#del brokerPerformance1
brokerPerformance1 = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos


In [92]:
dfprices2016 = pd.read_sql("SELECT * FROM nse2016", con=database_connection)
dfprices2016

,index,Symbol,Date,Closeval
0,0,20MICRONS,2016-01-01,35.20
1,1,3IINFOTECH,2016-01-01,5.15
2,2,3MINDIA,2016-01-01,11321.95
3,3,8KMILES,2016-01-01,2219.55
4,4,A2ZINFRA,2016-01-01,27.60
...,...,...,...,...
390782,390789,NIFTYHIGHBETA50,2016-12-30,1502.89
390783,390790,NIFTYLOWVOL50,2016-12-30,7758.30
390784,390791,NIFTY50VALUE20,2016-12-30,3652.00
390785,390792,NIFTYGROWSECT15,2016-12-30,5196.10


In [93]:
dfprices2017 = pd.read_sql("SELECT * FROM nse2017", con=database_connection)
dfprices2017

,index,Symbol,Date,Closeval
0,0,20MICRONS,2017-01-02,33.85
1,1,3IINFOTECH,2017-01-02,5.60
2,2,3MINDIA,2017-01-02,11056.45
3,3,8KMILES,2017-01-02,674.20
4,4,A2ZINFRA,2017-01-02,42.15
...,...,...,...,...
403386,403393,NIFTYHIGHBETA50,2017-12-29,2396.70
403387,403394,NIFTYLOWVOL50,2017-12-29,10102.21
403388,403395,NIFTY50VALUE20,2017-12-29,4732.00
403389,403396,NIFTYGROWSECT15,2017-12-29,6576.85


In [94]:
record = dfprices2016.loc[dfprices2016['Date'] == '2016-01-01']
v = record['Closeval'].iloc[0]
type(v)

numpy.float64

In [98]:
lost =[]
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['Analyst_or_Cmp'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(sym)

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['Call_Time_normal']#prediction date
            mydate1 = pd.Timestamp(mydate1)
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            mydate2 = pd.Timestamp(mydate2)
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
            #a = dfprices2016['Symbol']== sym
            #print(a)
            
            
            #Fetching current price from nse bav copy
            #cnx = mysql.connector.connect(user="root", password='yash', host="127.0.0.1", port=3306, database='mainschema')
            #sql_select_Query = "SELECT Closeval FROM "+dbyear1+" WHERE Date = '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            #cursor = cnx.cursor()
            #cursor.execute(sql_select_Query)
            #record = cursor.fetchall()
            #record = dfprices2016.loc[dfprices2016['Date'] == mydate1 & dfprices2016['Symbol']== sym]
            record = dfprices2016.loc[dfprices2016['Symbol']== sym]
            record = record.loc[record['Date']== mydate1]
            if len(record) != 0:
                
                x = record['Closeval'].iloc[0]
            
        
                #sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
                #dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
                #del dfprices1['index']
                dfprices1 = dfprices2016[dfprices2016['Date'] >= mydate1]
                del dfprices1['index']
        
                #sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
                #dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
                #del dfprices2['index']
                #dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)
                dfprices2 = dfprices2017[dfprices2017['Date'] <= mydate2]
                del dfprices2['index']
                dfprices1 = pd.concat([dfprices1,dfprices2],ignore_index=True)
                dfprices = dfprices1[dfprices1['Symbol'] == sym]
                #targetp = float(row['target_price'])

                targetp = float(row['Target_Price'])
        
                print("SYMBOL : ",sym)
                dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
                total = total + 1
            
                if not dfhits.empty :
                    print("HIT!")
                    hits = hits + 1
                    #recbuying = float(row['recomended_buying'])
                    maxval = dfprices['Closeval'].max()
                    growthperc = (maxval - x) *100 / x
                    print('Growth : ',growthperc)
                    growthSum = growthSum + growthperc
                    
                
                else:
                    print("DIDN'T HIT!")
                del dfprices
                del dfhits
            else:
                lost.append(sym)
    if total != 0: 
        hitRatio = int(hits*100 / total)
        print("Growthsum",growthSum)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformance1 = brokerPerformance1.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  Motilal Oswal
PETRONET
SYMBOL :  PETRONET
HIT!
Growth :  23.195735374521586
KSCL
SYMBOL :  KSCL
HIT!
Growth :  75.37707390648566
AUROPHARMA
SYMBOL :  AUROPHARMA
DIDN'T HIT!
AUROPHARMA
SYMBOL :  AUROPHARMA
DIDN'T HIT!
JAGRAN
SYMBOL :  JAGRAN
DIDN'T HIT!
JAGRAN
SYMBOL :  JAGRAN
DIDN'T HIT!
JKIL
SYMBOL :  JKIL
DIDN'T HIT!
PNCINFRA
SYMBOL :  PNCINFRA
HIT!
Growth :  92.57683215130024
JKIL
SYMBOL :  JKIL
HIT!
Growth :  56.02972908175495
JKIL
SYMBOL :  JKIL
DIDN'T HIT!
NATCOPHARM
SYMBOL :  NATCOPHARM
HIT!
Growth :  80.25094102885824
DLF
SYMBOL :  DLF
DIDN'T HIT!
CHENNPETRO
SYMBOL :  CHENNPETRO
HIT!
Growth :  70.06323396567298
AHLUCONT
SYMBOL :  AHLUCONT
DIDN'T HIT!
ITDCEM
SYMBOL :  ITDCEM
HIT!
Growth :  53.27305605786619
HAVELLS
SYMBOL :  HAVELLS
DIDN'T HIT!
SARLAPOLY
SYMBOL :  SARLAPOLY
DIDN'T HIT!
BRITANNIA
SYMBOL :  BRITANNIA
HIT!
Growth :  66.34469600177168
POWERGRID
SYMBOL :  POWERGRID
DIDN'T HIT!
CEATLTD
SYMBOL :  CEATLTD
HIT!
Growth :  57.08579580191563
TIINDIA
WABCOINDIA

SYMBOL :  SADBHAV
DIDN'T HIT!
NEULANDLAB
SYMBOL :  NEULANDLAB
HIT!
Growth :  50.55704414045973
PHILIPCARB
SYMBOL :  PHILIPCARB
HIT!
Growth :  223.57247437774524
VTL
SYMBOL :  VTL
HIT!
Growth :  35.163809617368244
BIOCON
SYMBOL :  BIOCON
HIT!
Growth :  32.09225886886548
BIOCON
SYMBOL :  BIOCON
HIT!
Growth :  32.09225886886548
POWERGRID
SYMBOL :  POWERGRID
DIDN'T HIT!
TAJGVK
SYMBOL :  TAJGVK
DIDN'T HIT!
UBL
SYMBOL :  UBL
HIT!
Growth :  20.34014881510661
DIVISLAB
SYMBOL :  DIVISLAB
DIDN'T HIT!
JETAIRWAYS
SYMBOL :  JETAIRWAYS
DIDN'T HIT!
BRITANNIA
SYMBOL :  BRITANNIA
HIT!
Growth :  31.304639677795723
BIRLACORPN
SYMBOL :  BIRLACORPN
DIDN'T HIT!
BIRLACORPN
SYMBOL :  BIRLACORPN
DIDN'T HIT!
ADANIPORTS
SYMBOL :  ADANIPORTS
HIT!
Growth :  73.01355578727842
ADANIPORTS
SYMBOL :  ADANIPORTS
HIT!
Growth :  73.01355578727842
LAXMIMACH
SYMBOL :  LAXMIMACH
HIT!
Growth :  43.65716606864802
DBCORP
SYMBOL :  DBCORP
DIDN'T HIT!
REDINGTON
SYMBOL :  REDINGTON
HIT!
Growth :  41.861575178997604
BYKE
SYMBOL :  

SYMBOL :  INDOCO
HIT!
Growth :  6.5881258719578355
REDINGTON
SYMBOL :  REDINGTON
DIDN'T HIT!
REDINGTON
SYMBOL :  REDINGTON
DIDN'T HIT!
REDINGTON
SYMBOL :  REDINGTON
DIDN'T HIT!
REDINGTON
SYMBOL :  REDINGTON
DIDN'T HIT!
PRICOLLTD
ASIANPAINT
SYMBOL :  ASIANPAINT
DIDN'T HIT!
FSL
SYMBOL :  FSL
DIDN'T HIT!
RALLIS
SYMBOL :  RALLIS
DIDN'T HIT!
RAMCOCEM
SYMBOL :  RAMCOCEM
DIDN'T HIT!
IDBI
SYMBOL :  IDBI
DIDN'T HIT!
INTELLECT
SYMBOL :  INTELLECT
DIDN'T HIT!
ZEEL
SYMBOL :  ZEEL
HIT!
Growth :  28.69052912405514
GENUSPOWER
SYMBOL :  GENUSPOWER
DIDN'T HIT!
MCDOWELL-N
SYMBOL :  MCDOWELL-N
HIT!
Growth :  8.610987409416635
HINDUNILVR
SYMBOL :  HINDUNILVR
HIT!
Growth :  21.162331630283326
NTPC
SYMBOL :  NTPC
HIT!
Growth :  25.708215297450433
NTPC
SYMBOL :  NTPC
HIT!
Growth :  25.132181882270007
HDFCBANK
SYMBOL :  HDFCBANK
HIT!
Growth :  27.96217770629236
COROMANDEL
SYMBOL :  COROMANDEL
DIDN'T HIT!
L&TFH
SYMBOL :  L&TFH
DIDN'T HIT!
L&TFH
SYMBOL :  L&TFH
DIDN'T HIT!
Growthsum 1673.9368984560074
Broker : 

SYMBOL :  DBCORP
DIDN'T HIT!
EXIDEIND
SYMBOL :  EXIDEIND
DIDN'T HIT!
ULTRACEMCO
SYMBOL :  ULTRACEMCO
HIT!
Growth :  32.602589877010395
RALLIS
SYMBOL :  RALLIS
DIDN'T HIT!
AUROPHARMA
SYMBOL :  AUROPHARMA
HIT!
Growth :  17.1401642655177
REPCOHOME
SYMBOL :  REPCOHOME
DIDN'T HIT!
KWALITY
SYMBOL :  KWALITY
DIDN'T HIT!
TATAPOWER
SYMBOL :  TATAPOWER
DIDN'T HIT!
SIYSIL
SYMBOL :  SIYSIL
HIT!
Growth :  55.234467983813374
Growthsum 1361.5452567870711
Broker :  Axis Direct HitRatio :  45 Avg. Growth :  17 Total Recos. :  80
brokerName :  Centrum
JKIL
SYMBOL :  JKIL
HIT!
Growth :  57.961165048543684
JKIL
SYMBOL :  JKIL
DIDN'T HIT!
COROMANDEL
SYMBOL :  COROMANDEL
HIT!
Growth :  86.38809182209468
ASHOKLEY
SYMBOL :  ASHOKLEY
HIT!
Growth :  69.18117343649257
BALKRISIND
SYMBOL :  BALKRISIND
HIT!
Growth :  80.96698311350083
BANKBARODA
SYMBOL :  BANKBARODA
DIDN'T HIT!
ASIANPAINT
SYMBOL :  ASIANPAINT
HIT!
Growth :  33.77522227546185
SITINET
SYMBOL :  SITINET
DIDN'T HIT!
HAVELLS
SYMBOL :  HAVELLS
HIT!
Growt

SYMBOL :  PNB
DIDN'T HIT!
TALBROAUTO
SYMBOL :  TALBROAUTO
DIDN'T HIT!
REPCOHOME
SYMBOL :  REPCOHOME
HIT!
Growth :  38.52854993268393
CARBORUNIV
SYMBOL :  CARBORUNIV
DIDN'T HIT!
CARBORUNIV
SYMBOL :  CARBORUNIV
DIDN'T HIT!
ASTRAMICRO
SYMBOL :  ASTRAMICRO
DIDN'T HIT!
BHARATFORG
SYMBOL :  BHARATFORG
HIT!
Growth :  43.825710754017294
ICICIBANK
SYMBOL :  ICICIBANK
DIDN'T HIT!
SADBHAV
SYMBOL :  SADBHAV
DIDN'T HIT!
CYIENT
MAGMA
SYMBOL :  MAGMA
DIDN'T HIT!
WIPRO
SYMBOL :  WIPRO
DIDN'T HIT!
CEATLTD
SYMBOL :  CEATLTD
HIT!
Growth :  31.629242574024367
HEIDELBERG
SYMBOL :  HEIDELBERG
DIDN'T HIT!
HCLTECH
SYMBOL :  HCLTECH
DIDN'T HIT!
GRASIM
SYMBOL :  GRASIM
HIT!
Growth :  39.60385580351249
GRASIM
SYMBOL :  GRASIM
HIT!
Growth :  39.60385580351249
ULTRACEMCO
SYMBOL :  ULTRACEMCO
HIT!
Growth :  29.537764207809396
ASIANPAINT
SYMBOL :  ASIANPAINT
HIT!
Growth :  35.275476733196186
AKZOINDIA
SYMBOL :  AKZOINDIA
HIT!
Growth :  34.553704138308696
ASTRAL
ASTRAL
PHOENIXLTD
SYMBOL :  PHOENIXLTD
DIDN'T HIT!
PHOE

SYMBOL :  ZEEL
HIT!
Growth :  28.695058073331815
INDIACEM
SYMBOL :  INDIACEM
DIDN'T HIT!
INDIACEM
SYMBOL :  INDIACEM
DIDN'T HIT!
ZUARI
SYMBOL :  ZUARI
DIDN'T HIT!
JAGRAN
SYMBOL :  JAGRAN
DIDN'T HIT!
JKIL
SYMBOL :  JKIL
HIT!
Growth :  94.97985745274251
GAIL
SYMBOL :  GAIL
HIT!
Growth :  32.63423878711307
NITINSPIN
SYMBOL :  NITINSPIN
HIT!
Growth :  91.00274725274728
IDBI
SYMBOL :  IDBI
DIDN'T HIT!
VSTTILLERS
SYMBOL :  VSTTILLERS
HIT!
Growth :  8.747680641084255
DIVISLAB
SYMBOL :  DIVISLAB
HIT!
Growth :  29.197220225257603
ICICIBANK
SYMBOL :  ICICIBANK
DIDN'T HIT!
SRTRANSFIN
SYMBOL :  SRTRANSFIN
HIT!
Growth :  34.92870106300233
PERSISTENT
INOXLEISUR
SYMBOL :  INOXLEISUR
DIDN'T HIT!
WABAG
SYMBOL :  WABAG
HIT!
Growth :  44.91717721806217
CROMPTON
SANDESH
SYMBOL :  SANDESH
HIT!
Growth :  44.74841902667035
ULTRACEMCO
SYMBOL :  ULTRACEMCO
HIT!
Growth :  47.23444599477659
KEC
SYMBOL :  KEC
DIDN'T HIT!
GPPL
SYMBOL :  GPPL
HIT!
Growth :  32.31500339443312
SHREECEM
SYMBOL :  SHREECEM
HIT!
Growth 

SYMBOL :  APOLLOTYRE
DIDN'T HIT!
Growthsum 0
Broker :  Investment Imperative HitRatio :  0 Avg. Growth :  0 Total Recos. :  1
brokerName :  Amit Gupta
ZEEL
SYMBOL :  ZEEL
HIT!
Growth :  14.397233201581031
JUBILANT
SYMBOL :  JUBILANT
DIDN'T HIT!
MARUTI
SYMBOL :  MARUTI
HIT!
Growth :  43.87091282724741
Growthsum 58.26814602882844
Broker :  Amit Gupta HitRatio :  66 Avg. Growth :  19 Total Recos. :  3
brokerName :  Systematix
NTPC
SYMBOL :  NTPC
HIT!
Growth :  6.4148681055155805
FSL
SYMBOL :  FSL
DIDN'T HIT!
PNB
SYMBOL :  PNB
DIDN'T HIT!
Growthsum 6.4148681055155805
Broker :  Systematix HitRatio :  33 Avg. Growth :  2 Total Recos. :  3
brokerName :  EIS
VIVIMEDLAB
SYMBOL :  VIVIMEDLAB
DIDN'T HIT!
JETAIRWAYS
SYMBOL :  JETAIRWAYS
HIT!
Growth :  15.86547249131785
GSPL
SYMBOL :  GSPL
HIT!
Growth :  47.21093461396381
SUNTECK
SYMBOL :  SUNTECK
HIT!
Growth :  45.98903836571999
VOLTAS
SYMBOL :  VOLTAS
HIT!
Growth :  75.8092738407699
JKIL
SYMBOL :  JKIL
DIDN'T HIT!
JKIL
SYMBOL :  JKIL
DIDN'T HIT!


SYMBOL :  DBCORP
DIDN'T HIT!
DBCORP
SYMBOL :  DBCORP
DIDN'T HIT!
JSWSTEEL
SYMBOL :  JSWSTEEL
HIT!
Growth :  41.0915839709113
BIOCON
SYMBOL :  BIOCON
HIT!
Growth :  90.24649036891935
GODREJPROP
SYMBOL :  GODREJPROP
HIT!
Growth :  59.4674120935707
VGUARD
SYMBOL :  VGUARD
HIT!
Growth :  89.08241291418861
ORIENTCEM
SYMBOL :  ORIENTCEM
DIDN'T HIT!
MCDOWELL-N
SYMBOL :  MCDOWELL-N
HIT!
Growth :  14.140108492964913
DHFL
SYMBOL :  DHFL
HIT!
Growth :  119.48944526264117
TVSMOTOR
SYMBOL :  TVSMOTOR
DIDN'T HIT!
SRTRANSFIN
SYMBOL :  SRTRANSFIN
HIT!
Growth :  37.99119690300683
FEDERALBNK
SYMBOL :  FEDERALBNK
DIDN'T HIT!
SRTRANSFIN
SYMBOL :  SRTRANSFIN
HIT!
Growth :  37.99119690300683
SADBHAV
SYMBOL :  SADBHAV
HIT!
Growth :  26.55303030303031
ICIL
SYMBOL :  ICIL
HIT!
Growth :  1.7186384194239424
ICICIBANK
SYMBOL :  ICICIBANK
DIDN'T HIT!
MARICO
SYMBOL :  MARICO
DIDN'T HIT!
DABUR
SYMBOL :  DABUR
DIDN'T HIT!
SHRIRAMCIT
SYMBOL :  SHRIRAMCIT
DIDN'T HIT!
INOXLEISUR
SYMBOL :  INOXLEISUR
DIDN'T HIT!
PERSISTE

SYMBOL :  GRASIM
HIT!
Growth :  40.99274511895873
Growthsum 40.99274511895873
Broker :  IES HitRatio :  100 Avg. Growth :  40 Total Recos. :  1
brokerName :  Fristcall
TITAN
SYMBOL :  TITAN
DIDN'T HIT!
KNRCON
SYMBOL :  KNRCON
HIT!
Growth :  61.91482935668982
Growthsum 61.91482935668982
Broker :  Fristcall HitRatio :  50 Avg. Growth :  30 Total Recos. :  2
brokerName :  PL
INDNIPPON
SYMBOL :  INDNIPPON
HIT!
Growth :  82.41206030150754
Growthsum 82.41206030150754
Broker :  PL HitRatio :  100 Avg. Growth :  82 Total Recos. :  1
brokerName :  Firstcall Research
HERITGFOOD
SYMBOL :  HERITGFOOD
HIT!
Growth :  121.14198994593569
ULTRACEMCO
SYMBOL :  ULTRACEMCO
HIT!
Growth :  48.517656399429825
COROMANDEL
SYMBOL :  COROMANDEL
DIDN'T HIT!
MHRIL
SYMBOL :  MHRIL
DIDN'T HIT!
Growthsum 169.6596463453655
Broker :  Firstcall Research HitRatio :  50 Avg. Growth :  42 Total Recos. :  4
brokerName :  SP Tulsian
ITDCEM
SYMBOL :  ITDCEM
DIDN'T HIT!
Growthsum 0
Broker :  SP Tulsian HitRatio :  0 Avg. Growt

In [99]:
len(lost)

47

In [100]:
total

1

In [101]:
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos
0,Motilal Oswal,41,21,109
1,Arihant Capital,50,32,2
2,ICICI Securties,0,0,1
3,AUM Capital,100,52,7
4,Edelweiss,56,26,120
...,...,...,...,...
89,Khambatta securities,50,46,2
90,Shubham Agarwal,50,4,2
91,Mayuresh Joshi,100,52,1
92,Simi Bhaumik,100,16,1


In [104]:
brokerPerformance1.to_sql(con=database_connection, name='mcbrokerperformance2016', if_exists='replace')